# FINAL PROJECT
## ANSHUMAN SARDA AND ARYAH RAO
ARYAH'S VERSION 

## CENTRAL IDEA

The aim of this project is to examine the quantities of contaminants present in various states of India by utilizing a dataset focused on Indian cities/states. The central question that we seek to address is as follows: What are states are affected by pollution in India? Is pollution correlated to population?

To answer our main question, we need to first clean and organize the data. Then, we will create graphs using tools like Matplotlib to show the levels of pollutants in different states and compare them. We will also use statistical analysis to identify any important differences between the states.

In [1]:
# Import Modules
datadir = "data"

import os
import io
import sys
import importlib 
import json
import pandas as pd
from lxml import etree
import requests
from IPython.display import Image
import sqlalchemy as sa
import sqlite3

module_path = os.path.abspath("modules")
if not module_path in sys.path:
    sys.path.append(module_path)

import util
importlib.reload(util)

htmlparser =  etree.HTMLParser()

%load_ext sql

Once we have imported all the required modules, we can begin working with the datasets.

## Parsing Datasets

Our intention is to employ a duo of datasets in this undertaking. The initial dataset, known as the Indian Cities Database, is obtainable at https://www.kaggle.com/datasets/kdsharmaai/india-city-air-quality-index?select=India_city_polution_data.xml. This dataset is suitable for retrieval and has been authorized under the Open Database License. The second dataset is a demographic dataset sourced from the Indian Census Bureau, and it can be accessed at https://en.wikipedia.org/api/rest_v1/page/html/List_of_states_and_union_territories_of_India_by_population. This dataset is similarly suitable for retrieval and is classified under the public domain.

Both of these datasets are eligible for scraping as they are publicly available on Kaggle and Wikipedia and do not have any restrictions on their use or distribution. We have reviewed the Terms of Service for Kaggle and Wikipedia API’s website and have ensured that we are following their guidelines for data scraping and use.

In order to obtain the necessary information, we will employ web scraping methodologies to extract the data from the websites and convert it into pandas DataFrames.

### AirQualityIndexes Table

The primary dataset we are utilizing is called "Indian States Air Quality," which is accessible via Kaggle. This dataset comprises details concerning Indian cities, including their respective state and city, as well as specifics on the quantities of different pollutants, such as NO2 CO, OZONE and so on.

We parse the XML file and store the root node in a variable named root.

In [2]:
# Parse pollution XML file
filepath = os.path.join(datadir, "pollution.xml")

stripparser = etree.XMLParser(remove_blank_text=True) 
tree = etree.parse(filepath, stripparser)
root = tree.getroot()

We make a list that records the details of all the values and then use it to make a DataFrame.

In [3]:
data = []

#Procedurally get data
for country in root.findall('Country'):
    for state in country.findall('State'):
        for city in state.findall('City'):
            for station in city.findall('Station'):
                details = {}
                for pollutant in station.findall('Pollutant_Index'):
                    pollutant_id = pollutant.get('id')
                    details[f'{pollutant_id}_Avg'] = pollutant.get('Avg')
                    details[f'{pollutant_id}_Max'] = pollutant.get('Max')
                    details[f'{pollutant_id}_Min'] = pollutant.get('Min')

                details['City'] = city.get('id')
                details['State'] = state.get('id')
                data.append(details)

In [4]:
# Raw Data
df = pd.DataFrame(data)
df

,PM2.5_Avg,PM2.5_Max,PM2.5_Min,PM10_Avg,PM10_Max,PM10_Min,NO2_Avg,NO2_Max,NO2_Min,NH3_Avg,...,SO2_Max,SO2_Min,CO_Avg,CO_Max,CO_Min,OZONE_Avg,OZONE_Max,OZONE_Min,City,State
0,30,70,10,43,88,12,19,33,7,3,...,108,13,29,41,2,11,63,1,Amaravati,Andhra_Pradesh
1,35,88,15,49,106,31,17,28,8,3,...,27,6,28,57,2,27,87,19,Rajamahendravaram,Andhra_Pradesh
2,122,179,60,79,100,45,31,48,9,2,...,8,4,49,74,44,33,39,15,Tirupati,Andhra_Pradesh
3,39,75,21,41,56,32,24,42,12,3,...,24,21,19,28,11,25,44,21,Visakhapatnam,Andhra_Pradesh
4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,Naharlagun,Arunachal_Pradesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,17,39,6,25,49,9,55,71,34,2,...,9,3,16,23,14,13,19,9,Kolkata,West_Bengal
304,32,51,20,52,76,35,17,22,14,4,...,23,11,23,30,20,19,34,11,Kolkata,West_Bengal
305,29,49,7,31,44,10,33,44,22,5,...,8,2,25,35,20,12,16,9,Kolkata,West_Bengal
306,14,27,4,14,35,3,20,25,16,6,...,16,6,17,25,10,13,22,9,Kolkata,West_Bengal


We tidy the DataFrame using melt and pivot a couple of times and lambda functions.

In [5]:
# Make DataFrame tidy
df = df.melt(id_vars=["City"], var_name="Elements", value_name="Values")
df = df.drop_duplicates(subset=['City', 'Elements'])
df = df.pivot(index="City", columns="Elements", values="Values")
df.reset_index(inplace=True)
df = df.melt(id_vars=["State", "City"], var_name="Elements", value_name="Values")
df["Quality"] = df["Elements"].apply(lambda x: x.split("_")[1])
df["Elements"] = df["Elements"].apply(lambda x: x.split("_")[0])
airqualityindexes = df.pivot(index=["State", "City", "Elements"], columns="Quality", values="Values")
airqualityindexes.sort_index(axis=1, level=0, inplace=True)
airqualityindexes.sort_index(axis=1, level=1, inplace=True)
airqualityindexes.reset_index(inplace=True)

# Display DataFrame 
airqualityindexes

Quality,State,City,Elements,Avg,Max,Min
0,Andhra_Pradesh,Amaravati,CO,29,41,2
1,Andhra_Pradesh,Amaravati,NH3,3,4,2
2,Andhra_Pradesh,Amaravati,NO2,19,33,7
3,Andhra_Pradesh,Amaravati,OZONE,11,63,1
4,Andhra_Pradesh,Amaravati,PM10,43,88,12
...,...,...,...,...,...,...
1122,West_Bengal,Siliguri,NO2,NA,NA,NA
1123,West_Bengal,Siliguri,OZONE,10,53,10
1124,West_Bengal,Siliguri,PM10,NA,NA,NA
1125,West_Bengal,Siliguri,PM2.5,NA,NA,NA


We clean up the DataFrame so that one state has one set of elements with average values.

In [6]:
# Remove rows with 'NA' values
airqualityindexes = airqualityindexes[airqualityindexes['Avg'] != 'NA']
airqualityindexes = airqualityindexes[airqualityindexes['Max'] != 'NA']
airqualityindexes = airqualityindexes[airqualityindexes['Min'] != 'NA']

# Convert the data type to float
airqualityindexes['Avg'] = airqualityindexes['Avg'].astype(float)
airqualityindexes['Max'] = airqualityindexes['Max'].astype(float)
airqualityindexes['Min'] = airqualityindexes['Min'].astype(float)

# Calculate mean values for each state
airqualityindexes = airqualityindexes.groupby(['State', 'Elements'], as_index=False).agg({'Avg': 'mean', 'Max': 'mean', 'Min': 'mean'})
airqualityindexes = airqualityindexes.drop(['Max', 'Min'], axis=1)

# Pivot Elements into columns
airqualityindexes['state'] = airqualityindexes['State'].str.replace('_', ' ')
airqualityindexes = airqualityindexes.pivot(index='state', columns='Elements', values='Avg')
airqualityindexes.reset_index(inplace=True)
airqualityindexes.dropna(inplace=True)

# Display DataFrame
airqualityindexes

Elements,state,CO,NH3,NO2,OZONE,PM10,PM2.5,SO2
0,Andhra Pradesh,31.250000,2.750000,22.750000,24.000000,53.000000,56.500000,17.750000
1,Assam,42.000000,3.000000,5.000000,10.000000,111.000000,164.000000,27.000000
2,Bihar,56.900000,10.307692,60.388889,23.000000,227.529412,270.529412,14.000000
3,Chandigarh,39.000000,5.000000,76.000000,15.000000,100.000000,110.000000,12.000000
5,Delhi,42.000000,1.000000,40.000000,5.000000,161.000000,220.000000,15.000000
6,Gujarat,68.800000,4.500000,51.666667,18.800000,132.000000,155.000000,27.000000
7,Haryana,39.791667,7.238095,34.958333,19.217391,131.090909,158.041667,12.391304
9,Karnataka,29.052632,3.937500,26.666667,20.400000,74.166667,72.388889,14.842105
10,Kerala,37.428571,2.142857,24.428571,21.600000,49.428571,44.142857,9.000000
11,Madhya Pradesh,35.538462,8.800000,37.357143,44.100000,145.500000,195.400000,25.000000


● state: The state in which the city resides. (STRING)
● CO: Average amount of CO in the air. (FLOAT)
● NH3: Average amount of NH3 in the air. (FLOAT)
● N02: Average amount of N02 in the air. (FLOAT)
● OZONE: Average amount of OZONE in the air. (FLOAT)
● PM10: Average amount of PM10 in the air. (FLOAT)
● PM2.5: Average amount of PM2.5 in the air. (FLOAT)
● S02: Average amount of S02 in the air. (FLOAT)

### Population Table

The secondary dataset we are employing is the Indian population index dataset, which is derived from the Indian government's census website. This dataset encompasses particulars regarding the population and percentage of national share for each Indian state and union territory.

We parse the html from the internet and parse the root node into a variable named pop_root.

In [7]:
# Read states from the web (Wikipedia's simpler API version)
resource_path = "/api/rest_v1/page/html/List_of_states_and_union_territories_of_India_by_population"

url = util.buildURL(resource_path, "en.wikipedia.org")
response = requests.get(url)
assert response.status_code == 200

# Use a custom HTML parser to parse the response content into an XML Element
tree = etree.parse(io.BytesIO(response.content), htmlparser)
pop_root = tree.getroot()

We construct a list of columns to make columns for a LoL.

In [8]:
# Read Table node
table_nodes = pop_root.xpath("//table")
table_node = table_nodes[1]

# Create header of columns
column_names = ["state", "population"]

We construct the LoL using xpath.

In [9]:
# Read state values
states = table_node.xpath("./tbody/tr/td[position()=2]/a/text()")

# Read population values
pop = table_node.xpath("./tbody/tr/td[position()=3]/span/text()")

# Create LoL of all the values
values = [states, pop]

We make a DataFrame using the LoL and the columns.

In [10]:
# Create DataFrame
DoL = {}
for i in range(2):
    DoL[column_names[i]] = values[i]

populations = pd.DataFrame(DoL)

#clean it out
populations['population'] = populations['population'].str.replace(',', '').astype(int)

# Display DataFrame
populations

,state,population
0,Uttar Pradesh,199812341
1,Maharashtra,112374333
2,Bihar,104099452
3,West Bengal,91276115
4,Madhya Pradesh,72626809
5,Tamil Nadu,72147030
6,Rajasthan,68548437
7,Karnataka,61095297
8,Gujarat,60439692
9,Andhra Pradesh,49577103


● state: Name of State or Union Territory. (STRING)
● population: The number of people in the state. (INT)

Inner join the twop tables: populations & airqualityindexes.

In [11]:
states = pd.merge(populations, airqualityindexes, on='state', how='inner')
states

,state,population,CO,NH3,NO2,OZONE,PM10,PM2.5,SO2
0,Uttar Pradesh,199812341,44.875000,11.357143,32.375000,10.941176,134.250000,183.312500,16.750000
1,Maharashtra,112374333,42.500000,3.285714,38.714286,34.111111,124.875000,129.666667,22.428571
2,Bihar,104099452,56.900000,10.307692,60.388889,23.000000,227.529412,270.529412,14.000000
3,West Bengal,91276115,35.200000,5.800000,29.800000,12.800000,54.200000,38.000000,13.333333
4,Madhya Pradesh,72626809,35.538462,8.800000,37.357143,44.100000,145.500000,195.400000,25.000000
5,Rajasthan,68548437,42.750000,11.000000,52.625000,28.500000,113.375000,144.125000,12.875000
6,Karnataka,61095297,29.052632,3.937500,26.666667,20.400000,74.166667,72.388889,14.842105
7,Gujarat,60439692,68.800000,4.500000,51.666667,18.800000,132.000000,155.000000,27.000000
8,Andhra Pradesh,49577103,31.250000,2.750000,22.750000,24.000000,53.000000,56.500000,17.750000
9,Odisha,41974219,46.000000,7.500000,24.000000,12.000000,44.000000,26.000000,28.000000


● state: Name of State or Union Territory. (STRING)
● population: The number of people in the state. (INT)
● CO: Average amount of CO in the air. (FLOAT)
● NH3: Average amount of NH3 in the air. (FLOAT)
● N02: Average amount of N02 in the air. (FLOAT)
● OZONE: Average amount of OZONE in the air. (FLOAT)
● PM10: Average amount of PM10 in the air. (FLOAT)
● PM2.5: Average amount of PM2.5 in the air. (FLOAT)
● S02: Average amount of S02 in the air. (FLOAT)

## SQL

We will store the DataFrames in a SQL database 'india.db'.

In [12]:
conn = sqlite3.connect('./dbfiles/india.db')
conn.close()

Establish connection to the SQL database.

In [13]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

def buildConnectionString(source="sqlite_india"):
    scheme, dbdir, database = getsqlite_creds(source=source)
    template = '{}:///{}/{}.db'
    return template.format(scheme, dbdir, database)

# Build the conection string
cstring = buildConnectionString("sqlite")
print("Connection string:", cstring)

# Connect to the database
engine = sa.create_engine(cstring)
connection = engine.connect()

Connection string: sqlite:///./dbfiles/india.db


Create table 'states' and store the dataframe into the table in india.db. Displays an error if table already exists.

In [14]:
statement = """
CREATE TABLE IF NOT EXISTS states (
    state VARCHAR(30),
    population VARCHAR(30),
    CO FLOAT,
    NH3 FLOAT,
    NO2 FLOAT,
    OZONE FLOAT,
    PM10 FLOAT,
    'PM2.5' FLOAT,
    SO2 FLOAT,
    PRIMARY KEY (state)
)
"""

# Execute the statement
try:
    connection.execute(statement)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of states failed:", str(err))

# Inserting directly from a DataFrame
try:
    states.to_sql("states", con=connection, if_exists="append", index=False)
except sa.exc.SQLAlchemyError as err:
    print("DataFrame to SQL failed:", str(err))

# Display SQL table as DataFrame
states = pd.read_sql_table("states", connection)
states

,state,population,CO,NH3,NO2,OZONE,PM10,PM2.5,SO2
0,Uttar Pradesh,199812341,44.875000,11.357143,32.375000,10.941176,134.250000,183.312500,16.750000
1,Maharashtra,112374333,42.500000,3.285714,38.714286,34.111111,124.875000,129.666667,22.428571
2,Bihar,104099452,56.900000,10.307692,60.388889,23.000000,227.529412,270.529412,14.000000
3,West Bengal,91276115,35.200000,5.800000,29.800000,12.800000,54.200000,38.000000,13.333333
4,Madhya Pradesh,72626809,35.538462,8.800000,37.357143,44.100000,145.500000,195.400000,25.000000
5,Rajasthan,68548437,42.750000,11.000000,52.625000,28.500000,113.375000,144.125000,12.875000
6,Karnataka,61095297,29.052632,3.937500,26.666667,20.400000,74.166667,72.388889,14.842105
7,Gujarat,60439692,68.800000,4.500000,51.666667,18.800000,132.000000,155.000000,27.000000
8,Andhra Pradesh,49577103,31.250000,2.750000,22.750000,24.000000,53.000000,56.500000,17.750000
9,Odisha,41974219,46.000000,7.500000,24.000000,12.000000,44.000000,26.000000,28.000000


In [15]:
# Close the connection
try:
    connection.close()
except:
    pass
del engine

## Challenges
The challenges we faced while storing data were:

1. airqualityindexes DataFrame was too messed up and we had to pivot, melt, drop rows multiple times to make it tidy.

2. populations DataFrame's populations column had to be modified to hold integer values without commas.

3. We needed to merge the two tables to make one entity and store it in the SQL database.

## Database Diagram

Good DB Design: The table adheres to good SQL database design principles by organizing data in rows and columns, with each row representing a unique record and each column representing a specific attribute or field. The table has a primary key column ("state") that uniquely identifies each row, ensuring data integrity and avoiding duplication. The column names are descriptive and represent the attributes of the data, and the data types used for each column are appropriate for the type of data being stored. The table also follows the normalization principle, as there are no redundant or duplicate data.

![states_table](./img.png)